# Word2Vec 개선 (1) embedding 계층
- 어휘가 100만개, 은닉층 뉴런이 100개인 CBOW 모델
    - 입력층과 출력층에는 각 100만개의 뉴런이 존재
        - 입력층의 원핫 표현과 가중치 행렬 W(in)의 곱 계산
        - 은닉층과 가중치 행렬 W(out) 의 곱 및 Softmax 계층의 계산

- 원핫 표현 문제
    - 어휘가 100만개라면 표현 하나만 해도 원소 수가 100만개인 벡터가 됨 -> 상당한 메모리 차지
    - W(in) 행렬과 곱할 때 계산 자원을 상당히 사용
        - **embedding 계층을 도입함으로써 해결**
- 은닉층 이후 계산
    - 은닉층과 가중치 행렬의 곱
        - **네거티브 샘플링이라는 새로운 손실 함수를 도입함으로써 해결**

### embedding 계층

In [1]:
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W, '\n')

# 특정 행을 출력하려면
print(W[2])
print(W[5],'\n')

# 여러 행을 한번에 출력하려면
idx = np.array([1,0,3,0])
print(W[idx])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]] 

[6 7 8]
[15 16 17] 

[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [1]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx): # 위에서 구현한 코드 그대로 복붙
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout): 
        '''
        embedding 계층의 순전파는 가중치 W의 특정 행을 추출할 뿐임
        따라서 역전파에서는 앞 층으로부터 전해진 기울기를 다음 층으로 그대로 흘러주면 됨
        다만, 앞 층으로부터 전해진 기울기를 가중치 기울기 dW의 특정 행에 설정함
        '''
        dW, = self.grads
        dW[...] = 0
        # dW[self.idx] = dout <- 나쁜 예시
        '''(나쁜 예시 코드 설명)
        가중치 기울기 dW를 꺼낸 다음 dW[...]=0 문장에서 dW의 원소를 0으로 덮어씀
        그리고 앞 층에서 전해진 기울기 dout을 idx번째 행에 할당'''
        np.add.at(dW, self.idx, dout)
        # for i, word_id in enumerate(self.idx): dW[word_id] += dout[i]
        return None 

# Word2Vec 개선 (2) 네거티브 샘플링
- 앞에서의 예에서 은닉층의 벡터 크기 100, 가중치 행렬의 크기가 100x100만
    - 행렬 곱 계산에 시간이 오래 걸림, 메모리도 많이 필요함
- Softmax에서도 같은 문제 발생
    - 어휘가 많아지면 계산량이 증가함

### 다중 분류에서 이진 분류로
- 다중 분류를 이진 분류로 근사하는 것
- 지금까지는 맥락이 주어졌을 때 정답이 되는 단어를 높은 확률로 추측하도록 만듦
    - 이제는 다중 분류 -> 이진 분류로
    - ex) 맥락이 you, goodbye일 때, 타깃 단어는 say입니까? 라는 질문에 답하는 신경망을 생각
    - 이렇게 하면 출력층에는 뉴런이 하나만 존재 -> say의 점수 출력
    - 최종 점수 : 1x100의 은닉층, 100x100만의 출력 가중치에서 say에 해당하는 단어 벡터 추출 -> 그 벡터와 은닉층 뉴런가의 내적

### 시그모이드 함수와 교차 엔트로피 오차
- 이진 분류 문제 -> 점수에 시그모이드 함수 적용 -> 확률로 변환 -> 손실 함수 : 교차 엔트로피 오차

### 다중 분류에서 이진 분류로 (구현)
- params : 매개변수 저장
- grads : 기울기 저장
- embed : enbedding 계층
- cache : 순전파 시의 계산 결과를 잠시 유지하기 위한 변수

In [2]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx): 
        # idx = 단어 ID 배열 (데이터를 한꺼번에 처리하는 미니배치 처리를 가정했기 때문)
        target_W = self.embed.forward(idx) # 벡터 불러오기
        out = np.sum(target_W * h, axis=1) # 내적 계산

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 네거티브 샘플링
- 맞는 경우 1에 가깝게, 이외의 단어일 경우에는 0에 가깝게 만듦
- 모든 부정적 예를 대상으로 하여 이진 분류를 학습?
    - 어휘 수가 늘어나면 감당 불가
    - **근사적 해법 : 부정적 예를 몇 개 선택 -> negative sampling**
    
    
- 정리
    - 네거티브 샘플링은 긍정적 예를 타깃으로 한 경우의 손실을 구하는 동시에 부정적 예 몇 개 샘플링하여 마찬가지로 손실을 구함
    - 각각의 데이터의 손실을 더한 값을 최종 손실로 함

### 네거티브 샘플링의 샘플링 기법
- 단순 무작위 샘플링?
    - 말뭉치의 통계 데이터를 기초로 샘플링!
    - 자주 등장하는 단어를 많이 추출하고, 드물게 등장하는 단어를 적게 추출!

In [3]:
import numpy as np
np.random.choice(10)

words = ['you','say','goodbye','I','hello','.']
np.random.choice(words)

# 5개 무작위 샘플링 (중복 있음)
np.random.choice(words, size=5)
# 5개 무작위 샘플링 (중복 없음)
np.random.choice(words, size=5, replace=False)
# 확률 분포에 따라 샘플링
p = [0.5,0.1,0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'I'

In [4]:
'''
기본 확률 분포에 0.75 제곱하기!
출현 확률이 낮은 단어를 버리지 않기 위해서
'''
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


- 유니그램 : 하나의 연속된 단어를 뜻함
- 바이그램, 트라이그램 ...
    - 따라서 유니그램 버전 : 한 단어를 대상으로 확률분포를 만든다는 의미

In [5]:
from common.np import * 
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample


In [6]:
corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2


# 긍정적 예로 3개의 데이터를 미니배치로 다룸
# 각각의 데이터에 대해 부정적 예를 2개씩 샘플링
# ex) 첫 번째 데이터에 대한 부정적 예 -> [0,3]
sample = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sample.get_negative_sample(target)
print(negative_sample)

[[3 0]
 [0 2]
 [2 3]]


### 네거티브 샘플링 구현

In [10]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

In [11]:
from common.np import *  # import numpy as np
from common.layers import Embedding

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [ ]:
import numpy as np
# from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
# if config.GPU:
#     contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)